In [29]:
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from pycocotools.coco import COCO

from PIL import Image
from pathlib import Path
from matplotlib.patches import Rectangle
from scipy.io import loadmat

import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random
from collections import Counter

import cv2
import matplotlib.pyplot as plt 
from datetime import datetime
from tqdm import tqdm

In [7]:
PATH = Path('data')
ANNOT_PATH = Path('data/annotations')

## COCO utils

In [8]:
def get_caption_index(annot_path, dataset='train', year=2017):
    annot_file = annot_path/f'captions_{dataset}{year}.json'
    return COCO(annot_file)

## Data

In [33]:
coco_train = get_caption_index(ANNOT_PATH, dataset='train')
coco_val = get_caption_index(ANNOT_PATH, dataset='val')

loading annotations into memory...
Done (t=1.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [41]:
def get_df(coco_api):
    annot_keys = list(coco_api.anns.keys())
    img_ids, img_paths, captions = [], [], []
    for k in tqdm(annot_keys):
        img_id = coco_api.anns[k]['image_id']
        img_ids.append(coco_api.anns[k]['image_id'])
        captions.append(coco_api.anns[k]['caption'])
        img_paths.append(coco_api.loadImgs(img_id)[0]['file_name'])
    df = pd.DataFrame({'img_id':img_ids, 'img_path':img_paths, 'caption':captions})
    return df

#### Datasets

In [42]:
df_train = get_df(coco_train)
df_val = get_df(coco_val)

100%|██████████| 25014/25014 [00:00<00:00, 325121.31it/s]


In [43]:
df_val

,img_id,img_path,caption
0,179765,000000179765.jpg,A black Honda motorcycle parked in front of a ...
1,179765,000000179765.jpg,A Honda motorcycle parked in a grass driveway
2,190236,000000190236.jpg,An office cubicle with four different types of...
3,331352,000000331352.jpg,A small closed toilet in a cramped space.
4,517069,000000517069.jpg,Two women waiting at a bench next to a street.
...,...,...,...
25009,9590,000000009590.jpg,A group of men sipping drinks and talking at a...
25010,84664,000000084664.jpg,"A plate of food with some eggs, potatoes, brea..."
25011,331569,000000331569.jpg,The strawberries was sitting beside the tall g...
25012,231237,000000231237.jpg,A bunch of small red flowers in a barnacle enc...
